In [1]:
import os, glob
# read a pos/neg dir
# each document is a review
corpus_folder = '/home/eric/Dropbox/490/final/review_polarity.v2/txt_sentoken/'
def readDir(senti_folder, pattern):
    file_list = []
    path_pattern = os.path.join(corpus_folder, senti_folder, pattern + '*.txt')
    all_txt_paths = glob.glob(path_pattern)
    # !!!!! I am only taking the first top_doc_num dcouments in pos/neg folder
    for file_path in all_txt_paths:
        word_List = readFile(file_path)
        file_list.append(word_List)
    return file_list

In [2]:
def readFile(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        tokenzied_words = f.read().split()
        return tokenzied_words

In [3]:
train_pos_file_list = readDir('pos', 'cv[0-8]')
train_neg_file_list = readDir('neg', 'cv[0-8]')
train_pos_labels = [1 for i in range(len(train_pos_file_list))]
train_neg_labels = [0 for i in range(len(train_neg_file_list))]

test_pos_file_list = readDir('pos', 'cv9')
test_neg_file_list = readDir('neg', 'cv9')
test_pos_labels = [1 for i in range(len(test_pos_file_list))]
test_neg_labels = [0 for i in range(len(test_neg_file_list))]

train_file_list = train_pos_file_list + train_neg_file_list
test_file_list = test_pos_file_list + test_neg_file_list

train_labels = train_pos_labels + train_neg_labels
test_labels = test_pos_labels + test_neg_labels

In [4]:
import string, re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [116]:
def clean(tokenized):
    punctuation_free = [x for x in tokenized if not re.fullmatch('[' + string.punctuation + ']+', x)]
#     unique_punctuation_free = set(punctuation_free)
    return ' '.join(punctuation_free)

## Problem 1 

M1
Unigrams (absence/presence)


In [117]:
train_file_list_clean = [clean(doc) for doc in train_file_list]
test_file_list_clean = [clean(doc) for doc in test_file_list]

In [118]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
# HashingVectorizer is not allowing get_feature_names method, idk why though

In [119]:
vectorizer = CountVectorizer(stop_words='english',binary=True)
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean])
test_features = vectorizer.transform([doc for doc in test_file_list_clean])

In [120]:
train_features.shape

(1800, 37679)

In [122]:
nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.85


In [123]:
import numpy as np
def show_most_informative_features(vectorizer, classifier, n=5):
    class_labels = classifier.classes_
    feature_names = vectorizer.get_feature_names()  
    topn_pos_class = sorted(zip(classifier.feature_count_[1], feature_names),reverse=True)[:n]
    topn_neg_class = sorted(zip(classifier.feature_count_[0], feature_names),reverse=True)[:n]    

    print("Important words in positive reviews")
    for coef, feature in topn_pos_class:
        print(class_labels[1], coef, feature) 
    print("-----------------------------------------")
    print("Important words in negative reviews")
    for coef, feature in topn_neg_class:
        print(class_labels[0], coef, feature)     
        
show_most_informative_features(vectorizer, nb_clf)

Important words in positive reviews
1 803.0 film
1 668.0 movie
1 653.0 like
1 578.0 just
1 576.0 time
-----------------------------------------
Important words in negative reviews
0 772.0 film
0 733.0 movie
0 693.0 like
0 612.0 just
0 554.0 time


M2
Unigrams with frequency count

In [124]:
vectorizer = CountVectorizer(stop_words='english',binary=False)
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean])
test_features = vectorizer.transform([doc for doc in test_file_list_clean])

nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.805


In [125]:
show_most_informative_features(vectorizer, nb_clf)

Important words in positive reviews
1 4703.0 film
1 2227.0 movie
1 1636.0 like
1 1200.0 just
1 1130.0 story
-----------------------------------------
Important words in negative reviews
0 3866.0 film
0 2900.0 movie
0 1690.0 like
0 1392.0 just
0 1043.0 time


M3
Unigrams (only adjectives/adverbs)

In [126]:
from nltk import pos_tag
def clean_adjv(tokenized):
    punctuation_free = [x for x in tokenized if not re.fullmatch('[' + string.punctuation + ']+', x)]
    word_posTags = pos_tag(punctuation_free, tagset='universal')
    adjv_words = [t[0] for t in word_posTags if t[1] in ("ADJ","ADV")]
    return ' '.join(adjv_words)

In [127]:
clean_adjv(["I","want" ,"this" ,"dilicious" ,"lunch",'badly'])

'dilicious badly'

In [128]:
train_file_list_clean_adjv = [clean_adjv(doc) for doc in train_file_list]
test_file_list_clean_adjv = [clean_adjv(doc) for doc in test_file_list]

In [129]:
vectorizer = CountVectorizer(stop_words='english',binary=False)
#,
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean_adjv])
test_features = vectorizer.transform([doc for doc in test_file_list_clean_adjv])

nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.85


In [130]:
show_most_informative_features(vectorizer, nb_clf)

Important words in positive reviews
1 1199.0 just
1 1099.0 good
1 729.0 best
1 693.0 really
1 691.0 little
-----------------------------------------
Important words in negative reviews
0 1391.0 just
0 1012.0 good
0 928.0 bad
0 709.0 really
0 660.0 little


M4
Unigrams (sublinear tf-idf)

In [134]:
vectorizer = TfidfVectorizer(stop_words='english', sublinear_tf=True)
#,max_features=10000
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean])
test_features = vectorizer.transform([doc for doc in test_file_list_clean])

nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.845


In [131]:
# TF-IDF
vectorizer = TfidfVectorizer(min_df = 5, stop_words='english', sublinear_tf=True, max_df=0.8)
#,max_features=10000
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean])
test_features = vectorizer.transform([doc for doc in test_file_list_clean])

nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.85


In [132]:
show_most_informative_features(vectorizer, nb_clf)

Important words in positive reviews
1 22.321209797866082 movie
1 18.722587051105048 like
1 16.861012933794072 story
1 16.81347161698053 life
1 16.742999677712586 just
-----------------------------------------
Important words in negative reviews
0 27.18739892096153 movie
0 21.17460434117034 like
0 19.76141374526718 just
0 18.672792925114685 bad
0 17.070786727241277 good


M5
Bigrams (absence/presence)

In [135]:
vectorizer = CountVectorizer(stop_words='english', ngram_range=(2,2),binary=True)
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean])
test_features = vectorizer.transform([doc for doc in test_file_list_clean])

In [136]:
train_features.shape

(1800, 448758)

In [137]:
nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.835


In [138]:
show_most_informative_features(vectorizer, nb_clf)

Important words in positive reviews
1 108.0 special effects
1 106.0 ve seen
1 75.0 year old
1 75.0 new york
1 63.0 takes place
-----------------------------------------
Important words in negative reviews
0 112.0 special effects
0 98.0 ve seen
0 71.0 new york
0 68.0 year old
0 68.0 looks like


## Problem 2

M1

In [113]:
from nltk.stem.porter import *
stemmer = PorterStemmer()


In [139]:
def clean(tokenized):
    punctuation_free = [x for x in tokenized if not re.fullmatch('[' + string.punctuation + ']+', x)]
#     unique_punctuation_free = set(punctuation_free)
    return ' '.join(punctuation_free)

In [140]:
train_file_list_clean = [clean(doc) for doc in train_file_list]
test_file_list_clean = [clean(doc) for doc in test_file_list]

In [141]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
# HashingVectorizer is not allowing get_feature_names method, idk why though

In [142]:
vectorizer = CountVectorizer(stop_words='english',binary=True)
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean])
test_features = vectorizer.transform([doc for doc in test_file_list_clean])

In [143]:
train_features.shape

(1800, 26894)

In [144]:
nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.865


In [145]:
show_most_informative_features(vectorizer, nb_clf)

Important words in positive reviews
1 847.0 thi
1 820.0 film
1 809.0 hi
1 742.0 ha
1 717.0 movi
-----------------------------------------
Important words in negative reviews
0 860.0 thi
0 799.0 film
0 774.0 hi
0 758.0 movi
0 713.0 ha


M2
Unigrams with frequency count

In [146]:
vectorizer = CountVectorizer(stop_words='english',binary=False)
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean])
test_features = vectorizer.transform([doc for doc in test_file_list_clean])

nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.815


In [147]:
show_most_informative_features(vectorizer, nb_clf)

Important words in positive reviews
1 5533.0 film
1 5020.0 hi
1 4158.0 thi
1 2660.0 movi
1 2329.0 ha
-----------------------------------------
Important words in negative reviews
0 4513.0 film
0 4420.0 thi
0 3594.0 hi
0 3246.0 movi
0 2199.0 wa


M3
Unigrams (only adjectives/adverbs)

In [148]:
from nltk import pos_tag
def clean_adjv(tokenized):
    punctuation_free = [x for x in tokenized if not re.fullmatch('[' + string.punctuation + ']+', x)]
    word_posTags = pos_tag(punctuation_free, tagset='universal')
    adjv_words = [stemmer.stem(t[0]) for t in word_posTags if t[1] in ("ADJ","ADV")]
    return ' '.join(adjv_words)

In [149]:
clean_adjv(["I","want" ,"this" ,"dilicious" ,"lunch",'badly'])

'dilici badli'

In [150]:
train_file_list_clean_adjv = [clean_adjv(doc) for doc in train_file_list]
test_file_list_clean_adjv = [clean_adjv(doc) for doc in test_file_list]

In [151]:
vectorizer = CountVectorizer(stop_words='english',binary=False)
#,
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean_adjv])
test_features = vectorizer.transform([doc for doc in test_file_list_clean_adjv])

nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.855


In [152]:
show_most_informative_features(vectorizer, nb_clf)

Important words in positive reviews
1 1199.0 just
1 1100.0 good
1 1025.0 veri
1 1013.0 onli
1 729.0 best
-----------------------------------------
Important words in negative reviews
0 1391.0 just
0 1203.0 onli
0 1012.0 good
0 928.0 bad
0 709.0 realli


M4
Unigrams (sublinear tf-idf)

In [153]:
# TF-IDF
vectorizer = TfidfVectorizer(min_df = 5, stop_words='english', sublinear_tf=True, max_df=0.8)
#,max_features=10000
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean])
test_features = vectorizer.transform([doc for doc in test_file_list_clean])

nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.85


In [154]:
show_most_informative_features(vectorizer, nb_clf)

Important words in positive reviews
1 22.82210910508472 wa
1 20.66562603602157 charact
1 20.59120438981204 like
1 19.767684246864327 make
1 19.341010803351747 time
-----------------------------------------
Important words in negative reviews
0 24.87719289173477 wa
0 23.455621053910058 like
0 21.271166445501652 charact
0 20.9847037027419 just
0 19.927320335592047 bad


M5
Bigrams (absence/presence)

In [155]:
vectorizer = CountVectorizer(stop_words='english', ngram_range=(2,2),binary=True)
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean])
test_features = vectorizer.transform([doc for doc in test_file_list_clean])

In [156]:
train_features.shape

(1800, 426650)

In [157]:
nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.835


In [158]:
show_most_informative_features(vectorizer, nb_clf)

Important words in positive reviews
1 374.0 thi film
1 229.0 thi movi
1 108.0 special effect
1 108.0 film wa
1 107.0 hi wife
-----------------------------------------
Important words in negative reviews
0 349.0 thi film
0 291.0 thi movi
0 145.0 look like
0 124.0 like thi
0 115.0 special effect


## Problem 3

In [324]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lexicon = pd.read_csv("NRC_Emotion.txt", sep='\t',header=None)

In [325]:
lexicon.columns = ['word','cat','flag']

In [326]:
lexicon_clean = lexicon[lexicon.cat.isin(['positive','negative'])].groupby("word",as_index=False).mean()
indicators = set(lexicon_clean[lexicon_clean.flag > 0]["word"])

In [327]:
lexicon_clean = lexicon[lexicon.word.isin(indicators)][lexicon.cat.isin(['positive','negative'])][lexicon.flag==1]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [328]:
lexicon_clean.flag = (lexicon_clean['cat'] == "positive")*1 - (lexicon_clean['cat'] == "negative")*1
lexicon_clean =lexicon_clean.groupby("word",as_index=False).mean()
lexicon_clean=lexicon_clean.reset_index()[['word','flag']]

In [329]:
vocabulary = {v: k for k, v in lexicon_clean['word'].to_dict().items()}

In [361]:
from nltk import pos_tag
def clean_lemma(tokenized):
    punctuation_free = [x for x in tokenized if not re.fullmatch('[' + string.punctuation + ']+', x)]
#     lemma_verb = [lemmatizer.lemmatize(word,'v') for word in punctuation_free]
#     lemma_noun = [lemmatizer.lemmatize(word,'n') for word in lemma_verb]
    word_posTags = pos_tag(punctuation_free, tagset='universal')
    adjv_words = [t[0] for t in word_posTags if t[1] in ("ADJ","ADV")]
    return ' '.join(adjv_words)

In [362]:
train_file_list_clean_lemma = [clean_lemma(doc) for doc in train_file_list]
test_file_list_clean_lemma = [clean_lemma(doc) for doc in test_file_list]
vectorizer = CountVectorizer(stop_words='english',binary=False , vocabulary=vocabulary)
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean_lemma])
test_features = vectorizer.transform([doc for doc in test_file_list_clean_lemma])

In [395]:
from sklearn.feature_extraction import DictVectorizer
lexicon_value = [{k:v} for k,v in lexicon_clean['flag'].to_dict().items()]
v = DictVectorizer(sparse=False)
X = v.fit_transform(lexicon_value)

threshold = test_features.dot(X).sum(axis=1).mean()
predictions = (test_features.dot(X).sum(axis=1) > threshold) * 1
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.7


## Problem 4

In [396]:
from nltk import pos_tag
def clean_adjv(tokenized):
    punctuation_free = [x for x in tokenized if not re.fullmatch('[' + string.punctuation + ']+', x)]
    word_posTags = pos_tag(punctuation_free, tagset='universal')
    adjv_words = [stemmer.stem(t[0]) for t in word_posTags if t[1] in ("ADJ","ADV")]
    return ' '.join(adjv_words)

In [397]:
train_file_list_clean_adjv = [clean_adjv(doc) for doc in train_file_list]
test_file_list_clean_adjv = [clean_adjv(doc) for doc in test_file_list]

In [398]:
vectorizer = CountVectorizer(stop_words='english',binary=False)
#,
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean_adjv])
test_features = vectorizer.transform([doc for doc in test_file_list_clean_adjv])

nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.855


In [405]:
vectorizer = TfidfVectorizer(min_df = 5, stop_words='english', sublinear_tf=True, max_df=0.8)
#,max_features=10000
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean_adjv])
test_features = vectorizer.transform([doc for doc in test_file_list_clean_adjv])

nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.85


In [407]:
vectorizer = CountVectorizer(stop_words='english')
#,max_features=10000
train_features = vectorizer.fit_transform([doc for doc in train_file_list_clean_adjv])
test_features = vectorizer.transform([doc for doc in test_file_list_clean_adjv])

nb_clf = MultinomialNB()
nb_clf.fit(train_features, train_labels)
predictions = nb_clf.predict(test_features)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, predictions)
print(accuracy)

0.855


In [419]:
import os, glob
# read a pos/neg dir
# each document is a review
corpus_folder = '/home/eric/Dropbox/490/final/review_polarity.v2/txt_sentoken/'
def readDir(senti_folder, pattern, label):
    file_list = []
    path_pattern = os.path.join(corpus_folder, senti_folder, pattern + '*.txt')
    all_txt_paths = glob.glob(path_pattern)
    # !!!!! I am only taking the first top_doc_num dcouments in pos/neg folder
    for file_path in all_txt_paths:
        doc = readFile(file_path)
        title = file_path.split("_")[-1].split(".")[0]
        file_list.append(label + " " + title + ": " + doc)
    return file_list

def readFile(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        tokenzied_words = " ".join(f.read().split('\n'))
        return tokenzied_words

In [425]:
train_fasttext_pos = readDir('pos', 'cv[0-8]','__label__1')
train_fasttext_neg = readDir('neg', 'cv[0-8]','__label__2')

test_fasttext_pos = readDir('pos', 'cv9','__label__1')
test_fasttext_neg = readDir('neg', 'cv9','__label__2')

In [427]:
with open("input_fasttext.txt",'w') as f:
    f.write("\n".join(train_fasttext_pos + train_fasttext_neg))
    
with open("test_fasttext.txt",'w') as f:
    f.write("\n".join(test_fasttext_pos + test_fasttext_neg))